<a href="https://colab.research.google.com/github/Akshay-s-kumar/3d-cnn-brain-tumor-segmentation/blob/main/LOad_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


third

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom

# Load train dataset
train_df = pd.read_csv("/content/drive/MyDrive/Brain Tumor Files/train.csv")
train_data_paths = train_df.iloc[:, 1:5].values.tolist()
train_mask_paths = train_df.iloc[:, 5].tolist()

test_df = pd.read_csv("/content/drive/MyDrive/Brain Tumor Files/test.csv")
test_data_paths = test_df.iloc[:, 1:5].values.tolist()
test_mask_paths = test_df.iloc[:, 5].tolist()

val_df = pd.read_csv("/content/drive/MyDrive/Brain Tumor Files/val.csv")
val_data_paths = val_df.iloc[:, 1:5].values.tolist()
val_mask_paths = val_df.iloc[:, 5].tolist()

IMG_SHAPE = (128, 128, 40, 4)
MASK_SHAPE = (128, 128, 40)

def load_nifti(path):
    path = path.decode('utf-8')
    img = nib.load(path).get_fdata()
    return np.array(img, dtype=np.float32)

def resize_volume(img, target_shape=IMG_SHAPE):
    factors = [target / img_dim for target, img_dim in zip(target_shape, img.shape)]
    return zoom(img, factors)

def resize_mask(mask, target_shape=MASK_SHAPE):
    factors = [target / img_dim for target, img_dim in zip(target_shape, mask.shape)]
    return zoom(mask, factors, order=0)

def load_and_preprocess_image(file_paths):
    images = [load_nifti(path) for path in file_paths]
    stacked_image = np.stack(images, axis=-1)
    stacked_image = stacked_image[:, :, 80:120, :]
    stacked_image = resize_volume(stacked_image)
    return stacked_image

def load_and_preprocess_mask(mask_path):
    mask = load_nifti(mask_path)
    mask = mask[:, :, 80:120]
    mask = resize_mask(mask)
    return mask

def ensure_image_shape(image):
    image.set_shape(IMG_SHAPE)
    return image

def ensure_mask_shape(mask):
    mask.set_shape(MASK_SHAPE)
    return mask

# Create train dataset (images)
train_dataset = tf.data.Dataset.from_tensor_slices(train_data_paths)
train_dataset = train_dataset.map(lambda x: tf.numpy_function(func=load_and_preprocess_image, inp=[x], Tout=tf.float32), num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
train_dataset = train_dataset.map(ensure_image_shape)
train_dataset = train_dataset.batch(1)

# Create train labels dataset (masks)
train_mask_dataset = tf.data.Dataset.from_tensor_slices(train_mask_paths)
train_mask_dataset = train_mask_dataset.map(lambda x: tf.numpy_function(func=load_and_preprocess_mask, inp=[x], Tout=tf.float32), num_parallel_calls=tf.data.AUTOTUNE)
train_mask_dataset = train_mask_dataset.prefetch(tf.data.AUTOTUNE)
train_mask_dataset = train_mask_dataset.map(ensure_mask_shape)
train_mask_dataset = train_mask_dataset.batch(1)

# Create test dataset (images)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data_paths)
test_dataset = test_dataset.map(lambda x: tf.numpy_function(func=load_and_preprocess_image, inp=[x], Tout=tf.float32), num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(ensure_image_shape)
test_dataset = test_dataset.batch(1)

# Create test labels dataset (masks)
test_mask_dataset = tf.data.Dataset.from_tensor_slices(test_mask_paths)
test_mask_dataset = test_mask_dataset.map(lambda x: tf.numpy_function(func=load_and_preprocess_mask, inp=[x], Tout=tf.float32), num_parallel_calls=tf.data.AUTOTUNE)
test_mask_dataset = test_mask_dataset.prefetch(tf.data.AUTOTUNE)
test_mask_dataset = test_mask_dataset.map(ensure_mask_shape)
test_mask_dataset = test_mask_dataset.batch(1)

# Create validation dataset (images)
val_dataset = tf.data.Dataset.from_tensor_slices(val_data_paths)
val_dataset = val_dataset.map(lambda x: tf.numpy_function(func=load_and_preprocess_image, inp=[x], Tout=tf.float32), num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.map(ensure_image_shape)
val_dataset = val_dataset.batch(1)

# Create validation labels dataset (masks)
val_mask_dataset = tf.data.Dataset.from_tensor_slices(val_mask_paths)
val_mask_dataset = val_mask_dataset.map(lambda x: tf.numpy_function(func=load_and_preprocess_mask, inp=[x], Tout=tf.float32), num_parallel_calls=tf.data.AUTOTUNE)
val_mask_dataset = val_mask_dataset.prefetch(tf.data.AUTOTUNE)
val_mask_dataset = val_mask_dataset.map(ensure_mask_shape)
val_mask_dataset = val_mask_dataset.batch(1)
